In [1]:
def savings(cls):
    cls.account_type = 'savings'
    return cls

def checking(cls):
    cls.account_type = 'checking'
    return cls

In [2]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

In [3]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'savings'})

In [4]:
Bank2Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'checking'})

In [5]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [6]:
@account_type('Savings')
class Bank1Savings:
    pass

@account_type('Checking')
class Bank1Checking:
    pass

In [7]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Savings' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Savings' objects>,
              '__doc__': None,
              'account_type': 'Savings'})

In [8]:
Bank1Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Checking' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Checking' objects>,
              '__doc__': None,
              'account_type': 'Checking'})

In [9]:
def hello(cls):
    cls.hello = lambda self: f'{self} says hello!'
    return cls

In [10]:
@hello
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name

In [11]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [12]:
p = Person('Guido')

In [13]:
p.hello()

'Guido says hello!'

In [14]:
from functools import wraps

In [15]:
def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

In [17]:
class Person:
    @func_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @func_logger
    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years old'

In [18]:
p = Person('John', 78)

Log: Person.__init__((<__main__.Person object at 0x000001BD736EBA88>, 'John', 78), {}) = None


In [19]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x000001BD736EBA88>,), {}) = Hello, my name is John, and I am 78 years old


'Hello, my name is John, and I am 78 years old'

In [20]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [21]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years old'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [22]:
p = Person('John', 78)

Log: Person.__init__((<__main__.Person object at 0x000001BD72D63AC8>, 'John', 78), {}) = None


In [23]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x000001BD72D63AC8>,), {}) = Hello, my name is John, and I am 78 years old


'Hello, my name is John, and I am 78 years old'

In [24]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print('static_method invoked...')
        
    @classmethod
    def cls_method(cls):
        print(f'cls_method invoked for {cls}...')
        
    def instance_method(self):
        print(f'instance_method invoked for {self}...')

decorating: <class '__main__.Person'> instance_method


In [25]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'static_method': <staticmethod at 0x1bd73635308>,
              'cls_method': <classmethod at 0x1bd73635848>,
              'instance_method': <function __main__.Person.instance_method(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [26]:
Person.__dict__['static_method']

In [27]:
callable(Person.__dict__['static_method'])

False

In [28]:
class Person:
    @staticmethod
    @func_logger
    def static_method():
        pass

In [29]:
Person.static_method()

Log: Person.static_method((), {}) = None


In [30]:
class Person:
    @func_logger
    @staticmethod
    def static_method():
        pass

In [31]:
Person.static_method()

TypeError: 'staticmethod' object is not callable

In [32]:
class Person:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def class_method(cls):
        pass

In [33]:
type(Person.__dict__['static_method'])

staticmethod

In [34]:
type(Person.__dict__['class_method'])

classmethod

In [35]:
Person.__dict__['static_method'].__func__

<function __main__.Person.static_method()>

In [36]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print('decorating static method', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print('decorating class method', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
    return cls

In [37]:
@class_logger
class Person:
    @staticmethod
    def static_method(a, b):
        print('static_method called...', a, b)
        
    @classmethod
    def class_method(cls, a, b):
        print('class_method called...', a, b)
        
    def instance_method(self, a, b):
        print('instance_method called...', a, b)

decorating static method <class '__main__.Person'> static_method
decorating class method <class '__main__.Person'> class_method
decorating callable <class '__main__.Person'> instance_method


In [38]:
Person.static_method(10, 20)

static_method called... 10 20
Log: Person.static_method((10, 20), {}) = None


In [39]:
Person.class_method(10, 20)

class_method called... 10 20
Log: Person.class_method((<class '__main__.Person'>, 10, 20), {}) = None


In [40]:
p = Person()

In [41]:
p.instance_method(10, 20)

instance_method called... 10 20
Log: Person.instance_method((<__main__.Person object at 0x000001BD73739F48>, 10, 20), {}) = None


In [42]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

decorating callable <class '__main__.Person'> __init__


In [43]:
type(Person.__dict__['name'])

property

In [44]:
isinstance(Person.__dict__['name'], property)

True

In [45]:
prop = Person.__dict__['name']

In [46]:
prop.fget

<function __main__.Person.name(self)>

In [47]:
prop.fset, prop.fdel

(None, None)

In [48]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print('decorating static method', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print('decorating class method', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', cls, name)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
    return cls

In [49]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

decorating callable <class '__main__.Person'> __init__
decorating property <class '__main__.Person'> name


In [50]:
p = Person('David')

Log: Person.__init__((<__main__.Person object at 0x000001BD7367AF08>, 'David'), {}) = None


In [51]:
p.name

Log: Person.name((<__main__.Person object at 0x000001BD7367AF08>,), {}) = David


'David'

In [52]:
@class_logger
class Person:
    class Other:
        def __call__(self):
            print('called instance of Other...')
    
    other = Other()

decorating callable <class '__main__.Person'> Other
decorating callable <class '__main__.Person'> other


In [53]:
Person.Other

<function __main__.Person.Other()>

In [54]:
Person.other

<function __main__.func_logger.<locals>.inner()>

In [55]:
import inspect 

In [56]:
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

In [57]:
keys = ('static_method', 'cls_method', 'inst_method', 'name', '__add__', 'Other', 'other')
inspect_funcs = ('isroutine', 'ismethod', 'isfunction', 'isbuiltin', 'ismethoddescriptor')

In [58]:
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_funcs)
print(format('', f'{max_fname_length}s'), '\t'.join(format(key, f'{max_header_length}s') for key in keys))
for inspect_func in inspect_funcs:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f'{max_header_length}s') for key in keys)
    print(format(inspect_func, f'{max_fname_length}s'), '\t'.join(inspect_results))

                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False        


In [59]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod):
            print('decorating static method', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print('decorating class method', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', cls, name)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
    return cls

In [61]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    @class_logger
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

decorating callable <class '__main__.MyClass.Other'> __call__
decorating static method <class '__main__.MyClass'> static_method
decorating class method <class '__main__.MyClass'> cls_method
decorating callable <class '__main__.MyClass'> inst_method
decorating property <class '__main__.MyClass'> name
decorating callable <class '__main__.MyClass'> __add__


In [64]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, (classmethod, staticmethod)):
            print(f'decorating {name}')
            type_ = type(obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', cls, name)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
    return cls

In [65]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    @class_logger
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

decorating callable <class '__main__.MyClass.Other'> __call__
decorating static_method
decorating cls_method
decorating callable <class '__main__.MyClass'> inst_method
decorating property <class '__main__.MyClass'> name
decorating callable <class '__main__.MyClass'> __add__


In [66]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, (classmethod, staticmethod)):
            print(f'decorating {name}')
            type_ = type(obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', cls, name)
            methods = (('fget', 'getter'), ('fset', 'setter'), ('fdel', 'deleter'))
            for prop, method in methods:
                if getattr(obj, prop):
                    obj = getattr(obj, method)(func_logger(getattr(obj, prop)))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
    return cls

In [67]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    @class_logger
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

decorating callable <class '__main__.MyClass.Other'> __call__
decorating static_method
decorating cls_method
decorating callable <class '__main__.MyClass'> inst_method
decorating property <class '__main__.MyClass'> name
decorating callable <class '__main__.MyClass'> __add__
